# Title

BIN membership metrics

## Purpose

HR Python Analysis - Data Request.


# Setup

## Library import

We import all the required Python libraries


In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import plotly
import plotly.graph_objs as go
import plotly.offline as ply

plotly.offline.init_notebook_mode(connected=True)

import cufflinks as cf

cf.go_offline(connected=True)
cf.set_config_file(theme="white")

import matplotlib as plt


## Local library import

We import all the required local libraries libraries


In [2]:
# Include local library paths
import datetime as dt
import sys

# Import local libraries Custom library
sys.path.append("../Library/")
import get_data as gd


# Parameter definition

We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the
other variables follow Python's guidelines.


In [21]:
# init get data class
ods = gd.Get_Data("postgres", "HRODS", "DEI")

# path definitions
ods.set_path(
    "input",
    "C:\\Users\\garciand\\OneDrive - HP Inc\\Desktop\\Deliverables\\Input\\"
)

ods.set_path(
    "output",
    "C:\\Users\\garciand\\OneDrive - HP Inc\\Desktop\\Deliverables\\Output\\DEI\\"
)

## Fiscal year setting.
## Uncomment the following line for a different fy start, Example FY starts in 11/01/2020
# ods.__set_fiscal_year(11, 1, 2020)

# Data import

We retrieve all the required data for the analysis.


In [25]:
# read input data
input_data = ods.file_query("BIN members 11-3-21.xlsx",read_sheet="Rawdata-Subscriptions")
field_Search = tuple(input_data["Email ID"].unique())

In [26]:
field_Search

('elizabeth.nguyen@hp.com',
 'emily.yen@hp.com',
 'ernescia.mitchell@hp.com',
 'eulanda.bryant@hp.com',
 'fiona.tao@hp.com',
 'francis.terence.tan@hp.com',
 'ghuang@hp.com',
 'han.wu@hp.com',
 'haoran.du@hp.com',
 'hinojosa@hp.com',
 'hitarthi.shah@hp.com',
 'howard.w.riley@hp.com',
 'hsin-fang.han@hp.com',
 'hsinlin.lai@hp.com',
 'huan.zhang@hp.com',
 'iryna.marchenko@hp.com',
 'james.knight@hp.com',
 'jason.canlas@hp.com',
 'jatin.verma@hp.com',
 'jesse.nasser@hp.com',
 'joel.stephen@hp.com',
 'joseph.zhao@hp.com',
 'joshua.park@hp.com',
 'justin.berry@hp.com',
 'kamil.ramazanov@hp.com',
 'kamioka@hp.com',
 'kang.yu@hp.com',
 'katherine.chung@hp.com',
 'kay.song@hp.com',
 'kelly.xie@hp.com',
 'kevin.mcilwain@hp.com',
 'lingzhang@hp.com',
 'lisa.zehm@hp.com',
 'lissa.zhang@hp.com',
 'lyna.chua@hp.com',
 'maria.skufati@hp.com',
 'marie.dieppe@hp.com',
 'matthew.wong@hp.com',
 'may.yee@hp.com',
 'michelle.lacasa@hp.com',
 'ming-chin.wu@hp.com',
 'muhammad.ali1@hp.com',
 'nicholas.b.hidd

# Data processing

Put here the core of the notebook. Feel free to further split this section into subsections.


In [17]:
# query additional fields
query_fields = """
    Select 
		wd."Email - Primary Work",
        wd."Business Lvl 1 (Group) Desc" As "L1 Business",
        wd."Management Chain - Level 01 Preferred Name" As "L1 Business Leader",
        wd."Business Lvl 2 (Unit) Desc" As "L2 Business",
        wd."Management Chain - Level 02 Preferred Name" As "L2 Business Leader",
        wd."Gender Code" As "Gender",
        Case
			When wd."Pay Group Country Desc" In (
			'Austria',
			'Belgium',
			'Bulgaria',
			'Croatia',
			'Czechia',
			'Denmark',
			'FinlAnd',
			'Greece',
			'Hungary',
			'IrelAnd',
			'Israel',
			'Kazakhstan',
			'Luxembourg',
			'Morocco',
			'NetherlAnds',
			'Nigeria',
			'Norway',
			'PolAnd',
			'Portugal',
			'Russian FederatiOn',
			'Saudi Arabia',
			'Serbia',
			'Slovakia',
			'South Africa',
			'Sweden',
			'Tunisia',
			'Turkey',
			'United Arab Emirates')
			Then 'U'
			When pwd."Employee ID" Is Null Then 'N'
			Else pwd."Is PWD?"
		End As "Disability",
        wd."Ethnicity",
        Case
			When vet."Employee ID" Is Null Then 'U'
			Else vet."Veteran Status"
		End As "US Veteran",
        wd."Management Level",
        wd."Is Supervisor" As "Is Manager",
        wd."Length of Service in Years" As "Tenure (in Years)",
        wd."Generation Category"
    From
        "HPW_DAILY" as wd
    Left Join
        "HPW_VETERANS" As vet On vet."Employee ID" = wd."Worker ID" and vet."Veteran Status" = 'Y' and vet."Report Date" = (Select Max("Report Date") From "HPW_VETERANS")
	Left Join
        "HPW_W_DISABILITIES" As pwd On pwd."Employee ID" = wd."Worker ID" and pwd."Report Date" = (Select Max("Report Date") From "HPW_W_DISABILITIES")
    Where
        wd."Email - Primary Work" In {0}
"""

# query database
join_data = ods.internal_query(query_fields.format(field_Search))

In [19]:
# join databases
output_data = pd.merge(input_data, join_data, left_on='Email ID', right_on="Email - Primary Work")

# remove unwanted column
output_data.drop('Email - Primary Work', inplace=True, axis=1)

# print sample output data
output_data.head()

,Employee ID,Email ID,Employee Name,Manager Email ID,City,Employee Status,Work\nStatus,Country,Business Group,Date of Registration,Month Number,Year Number,L1 Business,L1 Business Leader,L2 Business,L2 Business Leader,Gender,Disability,Ethnicity,US Veteran,Management Level,Is Manager,Tenure (in Years),Generation Category
0,90024590,elizabeth.nguyen@hp.com,Elizabeth Nguyen,andreig@hp.com,Spring,Active,Regular,United States,Personal Systems,11/14/2019,November,2019.0,Personal Systems,"Cho, Alex",PS SW Engineering,"Norlander, Rebecca",F,N,Asian (United States of America),U,INT,N,2,Generation Y
1,90024590,elizabeth.nguyen@hp.com,Elizabeth Nguyen,andreig@hp.com,Spring,Active,Regular,United States,Personal Systems,11/14/2019,November,2019.0,Personal Systems,"Cho, Alex",PS SW Engineering,"Norlander, Rebecca",F,N,Asian (United States of America),U,INT,N,2,Generation Y
2,90024590,elizabeth.nguyen@hp.com,Elizabeth Nguyen,andreig@hp.com,Spring,Active,Regular,United States,Personal Systems,11/14/2019,November,2019.0,Personal Systems,"Cho, Alex",PS SW Engineering,"Norlander, Rebecca",F,N,Asian (United States of America),U,INT,N,2,Generation Y
3,90024590,elizabeth.nguyen@hp.com,Elizabeth Nguyen,andreig@hp.com,Spring,Active,Regular,United States,Personal Systems,11/14/2019,November,2019.0,Personal Systems,"Cho, Alex",PS SW Engineering,"Norlander, Rebecca",F,N,Asian (United States of America),U,INT,N,2,Generation Y
4,90024590,elizabeth.nguyen@hp.com,Elizabeth Nguyen,andreig@hp.com,Spring,Active,Regular,United States,Personal Systems,11/20/2019,November,2019.0,Personal Systems,"Cho, Alex",PS SW Engineering,"Norlander, Rebecca",F,N,Asian (United States of America),U,INT,N,2,Generation Y


In [23]:
# export final report
ods.export_data(output_data, custom_filename="BIN members 11-3-21")